In [9]:
%matplotlib inline
from fancy_plot import *
import numpy as np
import pickle
from scipy.optimize import curve_fit
import glob
import pandas as pd
import datetime
import pytz
from tzwhere import tzwhere

In [20]:
def make_prediction_for_site(site, target_key, res_gpp, res_nee):
    spred, targets, tmps_return = None, None, None
    for f in glob.glob(folder):
        with open(f, 'rb') as ifile:
            idata = pickle.load(ifile)
        if site not in idata.keys():
            continue
        ipt = np.array(idata[site]['input_data'])
        tmsp = np.array(idata[site]['input_data_timestamps'])
        flxnet = idata[site]['fluxnet_data']
        targets = []
        inputs = []
        tmps_return = []
        for c, t in enumerate(tmsp):
            if (t.hour % 1 != 0):
                continue
            try:
                a = float(flxnet.loc[flxnet['datetime_utc'] == t][target_key].iloc[0])
                if a < -1000:
                    continue
            except:
                continue
            targets.append(a)
            inputs.append(ipt[c])
            tmps_return.append(t)
        tmps_return = np.array(tmps_return)   
        inputs = np.array(inputs)
        inputs[:,2] = inputs[:,2] 
        targets = np.array(targets)
        if len(inputs) == 0:
            print('Nothing to plot. continue')
            return None, None, None
    #    print(np.min(inputs[:,1]), np.min(inputs[:,4]), np.min(inputs[:,3]), np.min(inputs[:,0]), np.min(inputs[:,2]))
        if 'NEE' in target_key:
            spred = -(res_gpp[0][0] * inputs[:,1] * inputs[:,4]  * inputs[:,3]) * inputs[:,0] * inputs[:,2] / (1 + inputs[:,2]/res_gpp[0][1]) + res_nee[0][0] * (inputs[:,5])  + res_nee[0][1]
        else:
            spred = (res_gpp[0][0] * inputs[:,1] * inputs[:,4]  * inputs[:,3]) * inputs[:,0] * inputs[:,2] / (1 + inputs[:,2]/res_gpp[0][1])
            targets[targets<0] = 0
    return spred, targets, tmps_return


In [3]:
# Index(['TIMESTAMP_START', 'TIMESTAMP_END', 'NEE_CUT_REF', 'NEE_VUT_REF',
#        'NEE_CUT_REF_QC', 'NEE_VUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_NT_CUT_REF',
#        'GPP_DT_VUT_REF', 'GPP_DT_CUT_REF', 'datetime_utc'],
#       dtype='object')

In [4]:
site_dict = {'grassland': ['AT-Neu', 'CH-Cha', 'CH-Fru', 'CZ-BK2', 'DE-Gri', 'DE-RuR', 'IT-MBo', 'IT-Tor'],
             'mixed_forests': ['BE-Bra', 'BE-Vie', 'CH-Lae'],
             'cropland': ['BE-Lon', 'CH-Oe2', 'DE-Geb', 'DE-Kli', 'DE-RuS', 'FR-Gri', 'IT-BCi', 'IT-CA2'],
             'evergeen': ['CH-Dav', 'CZ-BK1', 'DE-Lkb', 'DE-Obe', 'DE-Tha', 'FI-Hyy', 'FI-Let', 'FI-Sod',
                          'IT-Lav', 'IT-Ren', 'IT-SRo', 'NL-Loo', 'RU-Fyo'],
       #      'wetland': ['CZ-wet', 'DE-Akm', 'DE-SfN', 'DE-Spw', 'FR-Pue', 'IT-Cp2'],
             'deciduous': ['DE-Hai', 'DE-Lnf', 'DK-Sor', 'FR-Fon', 'IT-CA1', 'IT-CA3', 'IT-Col', 'IT-Ro2'],
             'shrubland': ['ES-Amo', 'ES-LJu', 'IT-Noe']}

In [5]:
veg_type_id = {'grassland': 7, 'mixed_forests': 3 , 'cropland':6, 
               'evergeen': 1, 'deciduous': 2, 'shrubland':4} # 'wetland': 8

In [6]:
viirs_dict = dict()

In [7]:
### target_key  = 'NEE_st_fMDS' # #'GPP_st_MDS' #

do_checks = False
fit_res_dict_viirs = dict()
folder = '/work/bd1231/tglauch/VPRM_output_viirs_new_sites/*.pickle'
for veg_type in site_dict.keys():
    print(veg_type)
    targets_gpp = []
    targets_nee = []
    inputs = []
    lens = [0]
    len0 = 0
    for f in glob.glob(folder):
        with open(f, 'rb') as ifile:
            idata = pickle.load(ifile)
        for site in site_dict[veg_type]:
            if site not in idata.keys():
                continue
            tzw = tzwhere.tzwhere()
            timezone_str = tzw.tzNameAt(idata[site]['lonlat'][1], idata[site]['lonlat'][0]) # Seville coordinates
            timezone = pytz.timezone(timezone_str)
            dt = datetime.datetime.now()
            if do_checks:
                targets = []
                inputs = []
            print('Use {} at {} (UTC Offset {})'.format(site, idata[site]['lonlat'],
                                                        timezone.utcoffset(dt).total_seconds()/3600))
            ipt = np.array(idata[site]['input_data'])
            tmsp = np.array(idata[site]['input_data_timestamps'])
            flxnet = idata[site]['fluxnet_data']
            for c, t in enumerate(tmsp):
                try:
                    if float(flxnet.loc[flxnet['datetime_utc'] == t]['NEE_VUT_REF_QC'])>1:
                        continue
                    a = float(flxnet.loc[flxnet['datetime_utc'] == t]['NEE_VUT_REF'])
                    b = float(flxnet.loc[flxnet['datetime_utc'] == t]['GPP_DT_VUT_REF'])
                except Exception as e :
                    print(e)
                    continue
                targets_nee.append(a)
                inputs.append(ipt[c])
                targets_gpp.append(b)
            lens.append(len(targets_gpp)-len0)
            len0 = len(targets_gpp)
    inds = []
    lens = np.array(lens)
    if np.sum(lens) == 0:
        continue
    num_inds_per_site = np.min(lens[lens>0])
    cum_len = np.cumsum(lens)
    for i in range(len(cum_len)-1):
        if cum_len[i] == cum_len[i+1]:
            continue
        inds.append(np.random.choice(np.arange(cum_len[i], cum_len[i+1]),
                                     num_inds_per_site))
    inds = np.concatenate(inds)  
    inputs = np.array(inputs)[inds]
    targets_gpp = np.array(targets_gpp)[inds]
    targets_nee = np.array(targets_nee)[inds]
    mask = targets_gpp > 0.5
    mse_min = 1e4
    # inputs[:,0][inputs[:,0]<0] = 0 
    for i in range(100):
        res_gpp_t = curve_fit(lambda x, lamb, par0: (lamb * x[:,1] * x[:,4]  * x[:,3]) * x[:,0] * x[:,2] / (1 + x[:,2]/par0),
                        inputs[mask], targets_gpp[mask], maxfev=5000, p0=[np.random.uniform(0,1), np.random.uniform(0,1)]) 
        spred = (res_gpp_t[0][0] * inputs[:,1] * inputs[:,4]  * inputs[:,3]) * inputs[:,0] * inputs[:,2] / (1 + inputs[:,2]/res_gpp_t[0][1])
        mse = (np.square(targets_gpp[mask] - spred[mask])).mean()
        if mse < mse_min:
            res_gpp = res_gpp_t
            mse_min = mse
    mse_min = 1e4
    print('-----')
    for i in range(100):
        res_nee_t = curve_fit(lambda x, a,b : -(res_gpp[0][0] * x[:,1] * x[:,4]  * x[:,3]) * x[:,0] * x[:,2] / (1 + x[:,2]/res_gpp[0][1]) + a * x[:,5] + b,
                        inputs, targets_nee, maxfev=5000, p0=[np.random.uniform(0,1), np.random.uniform(0,1)])  
        spred = -(res_gpp[0][0] * inputs[:,1] * inputs[:,4]  * inputs[:,3]) * inputs[:,0] * inputs[:,2] / (1 + inputs[:,2]/res_gpp[0][1]) + res_nee_t[0][0] * (inputs[:,5])  + res_nee_t[0][1]
        mse = (np.square(targets_nee - spred)).mean()
        if mse < mse_min:
            res_nee = res_nee_t
            mse_min = mse
    print(res_gpp, res_nee)
    fit_res_dict_viirs[veg_type_id[veg_type]] = np.concatenate([res_gpp[0], res_nee[0]])
    print(fit_res_dict_viirs)
    target_key  = 'GPP_DT_VUT_REF' #'NEE_VUT_REF' 
    for site in site_dict[veg_type]:
            spred, targets, tmsp = make_prediction_for_site(site, target_key, res_gpp, res_nee)
            if spred is None:
                continue
#             ax_max = np.max([spred, targets])
#             ax_min = np.min([spred, targets])
#             fig, ax = newfig(0.9,ratio=1.0)
#             ax.scatter(targets, spred, s=3)
#             ax.set_xlabel('Target')
#             ax.set_ylabel('Prediction')
#             ax.text(0.1, 0.9, site,
#                  horizontalalignment='left',
#                  verticalalignment='top',
#                  transform = ax.transAxes)
#             ax.plot([ax_min, ax_max], [ax_min, ax_max], color='k',
#                     lw=2)
#             fig.show()

#             fig, ax = newfig(2.0, ratio=0.3)
#             ax.plot(range(len(targets)), targets, lw=1, color='k',
#                     marker='o', ms=1)
#             ax.plot(range(len(spred)), spred, lw=1,
#                     marker='o', ms=1)
#             ax.text(0.1, 0.9, site,
#                  horizontalalignment='left',
#                  verticalalignment='top',
#              transform = ax.transAxes)
#             # ax.set_xlim(500, 1800)
#             fig.show()
            # month_values_pred = []
            # temp_values_pred = [] 
            # month_values_truth = []
            # temp_values_truth = [] 
            # c_month = tmsp[0].month
            # for c, i in enumerate(tmsp):
            #     if i.month != c_month:
            #         c_month = i.month
            #         month_values_pred.append(np.mean(temp_values_pred))
            #         month_values_truth.append(np.mean(temp_values_truth))
            #         temp_values_truth = [] 
            #         temp_values_pred = []
            #     temp_values_pred.append(spred[c])
            #     temp_values_truth.append(targets[c])
            # fig, ax = newfig(0.9)
            # ax.plot(range(len(month_values_truth)), month_values_truth, color='k',
            #         marker='o')
            # ax.plot(range(len(month_values_pred)), month_values_pred, color='b',
            #         marker='o')
            # ax.text(0.0, 0.9, site,
            #      horizontalalignment='left',
            #      verticalalignment='top',
            #      transform = ax.transAxes)
            # fig.show()
            viirs_dict[site] = [spred, targets, tmsp]

grassland


/home/b/b309233/.local/lib/python3.9/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


Use AT-Neu at (11.3175, 47.11667) (UTC Offset 2.0)
cannot convert the series to <class 'float'>
Use CH-Cha at (8.41044, 47.21022) (UTC Offset 2.0)
Use CH-Fru at (8.53778, 47.11583) (UTC Offset 2.0)
Use IT-MBo at (11.04583, 46.01468) (UTC Offset 2.0)
Use IT-Tor at (7.57806, 45.84444) (UTC Offset 2.0)
Use DE-Gri at (13.51253, 50.94947) (UTC Offset 2.0)
Use DE-RuR at (6.30413, 50.62191) (UTC Offset 2.0)
Use CZ-BK2 at (18.54285, 49.49443) (UTC Offset 2.0)
cannot convert the series to <class 'float'>
-----
(array([4.36418969e-02, 5.40139731e+02]), array([[ 1.74395882e-06, -3.34929103e-02],
       [-3.34929103e-02,  6.65430180e+02]])) (array([0.35125761, 0.63466387]), array([[ 1.92186192e-05, -1.73495567e-04],
       [-1.73495567e-04,  2.39700707e-03]]))
{7: array([4.36418969e-02, 5.40139731e+02, 3.51257606e-01, 6.34663865e-01])}
mixed_forests
Use CH-Lae at (8.36439, 47.47833) (UTC Offset 2.0)
Use BE-Bra at (4.51984, 51.30761) (UTC Offset 2.0)
Use BE-Vie at (5.99812, 50.30493) (UTC Offset 2.

In [17]:
fit_res_dict_viirs

{7: array([4.39053678e-02, 5.39511331e+02, 3.52112462e-01, 6.48314796e-01]),
 3: array([4.33352471e-02, 4.28541153e+02, 2.66433468e-01, 1.05088322e+00]),
 6: array([4.18927109e-02, 5.39236756e+02, 1.48111163e-01, 3.90592341e-01]),
 1: array([4.48877698e-02, 4.32558117e+02, 2.73889237e-01, 1.29642867e-01]),
 2: array([4.13878378e-02, 4.53203997e+02, 7.57708450e-02, 1.90555458e+00]),
 4: array([2.82067925e-02, 3.45295752e+02, 6.48748662e-02, 3.10600464e-01])}

In [9]:
with open('/home/b/b309233/fit_res_viirs.pickle', 'w+b') as ofile:
    pickle.dump(fit_res_dict_viirs, ofile)

In [25]:
modis_dict = dict()

In [29]:
### target_key  = 'NEE_st_fMDS' # #'GPP_st_MDS' #

do_checks = False
fit_res_dict_modis = dict()
folder = '/work/bd1231/tglauch/VPRM_output_modis_new_sites_new/*.pickle'
for cvt, veg_type in enumerate(site_dict.keys()):
    # if cvt > 0:
    #     continue
    print(veg_type)
    targets_gpp = []
    targets_nee = []
    inputs = []
    lens = [0]
    len0 = 0
    for f in glob.glob(folder):
        with open(f, 'rb') as ifile:
            idata = pickle.load(ifile)
        for site in site_dict[veg_type]:
            if site not in idata.keys():
                continue
            tzw = tzwhere.tzwhere()
            timezone_str = tzw.tzNameAt(idata[site]['lonlat'][1], idata[site]['lonlat'][0]) # Seville coordinates
            timezone = pytz.timezone(timezone_str)
            dt = datetime.datetime.now()
            if do_checks:
                targets = []
                inputs = []
            print('Use {} at {} (UTC Offset {})'.format(site, idata[site]['lonlat'],
                                                        timezone.utcoffset(dt).total_seconds()/3600))
            ipt = np.array(idata[site]['input_data'])
            tmsp = np.array(idata[site]['input_data_timestamps'])
            flxnet = idata[site]['fluxnet_data']
            for c, t in enumerate(tmsp):
                try:
                    if float(flxnet.loc[flxnet['datetime_utc'] == t]['NEE_VUT_REF_QC'].iloc[0])>1:
                        continue
                    a = float(flxnet.loc[flxnet['datetime_utc'] == t]['NEE_VUT_REF'].iloc[0])
                    b = float(flxnet.loc[flxnet['datetime_utc'] == t]['GPP_DT_VUT_REF'].iloc[0])
                except Exception as e :
                    print(e)
                    continue
                targets_nee.append(a)
                inputs.append(ipt[c])
                targets_gpp.append(b)
            lens.append(len(targets_gpp)-len0)
            len0 = len(targets_gpp)
    inds = []
    lens = np.array(lens)
    if np.sum(lens) == 0:
        continue
    num_inds_per_site = np.min(lens[lens>0])
    cum_len = np.cumsum(lens)
    for i in range(len(cum_len)-1):
        if cum_len[i] == cum_len[i+1]:
            continue
        inds.append(np.random.choice(np.arange(cum_len[i], cum_len[i+1]),
                                     num_inds_per_site))
    inds = np.concatenate(inds)  
    inputs = np.array(inputs)[inds]
    targets_gpp = np.array(targets_gpp)[inds]
    targets_nee = np.array(targets_nee)[inds]
    mask = targets_gpp > 0.5
    mask_nan = np.all(np.isfinite(inputs), axis=1)
    mse_min = 1e4
    # inputs[:,0][inputs[:,0]<0] = 0 
    print(len(inputs[mask]))
    print(np.max(inputs[:,0]))
    for i in range(100):
        res_gpp_t = curve_fit(lambda x, lamb, par0: (lamb * x[:,1] * x[:,4]  * x[:,3]) * x[:,0] * x[:,2] / (1 + x[:,2]/par0),
                        inputs[mask_nan & mask], targets_gpp[mask_nan & mask], maxfev=5000, p0=[np.random.uniform(0,1), np.random.uniform(0,1)]) 
        spred = (res_gpp_t[0][0] * inputs[:,1] * inputs[:,4]  * inputs[:,3]) * inputs[:,0] * inputs[:,2] / (1 + inputs[:,2]/res_gpp_t[0][1])
        mse = (np.square(targets_gpp[mask_nan & mask] - spred[mask_nan & mask])).mean()
        if mse < mse_min:
            res_gpp = res_gpp_t
            mse_min = mse
    print(mse_min)
    mse_min = 1e4
    print('-----')
    for i in range(3):
        res_nee_t = curve_fit(lambda x, a,b : -(res_gpp[0][0] * x[:,1] * x[:,4]  * x[:,3]) * x[:,0] * x[:,2] / (1 + x[:,2]/res_gpp[0][1]) + a * x[:,5] + b,
                        inputs[mask_nan], targets_nee[mask_nan], maxfev=5000, p0=[np.random.uniform(0,1), np.random.uniform(0,1)])  
        spred = -(res_gpp[0][0] * inputs[:,1] * inputs[:,4]  * inputs[:,3]) * inputs[:,0] * inputs[:,2] / (1 + inputs[:,2]/res_gpp[0][1]) + res_nee_t[0][0] * (inputs[:,5])  + res_nee_t[0][1]
        mse = (np.square(targets_nee[mask_nan] - spred[mask_nan])).mean()
        if mse < mse_min:
            res_nee = res_nee_t
            mse_min = mse
    print(mse_min)
    print(res_gpp, res_nee)
    fit_res_dict_modis[veg_type_id[veg_type]] = [res_gpp, res_nee]
    continue
    print('Make plots')
    target_key  = 'GPP_DT_VUT_REF' #'NEE_VUT_REF' 
    
    for site in site_dict[veg_type]:
        spred, targets, tmsp = make_prediction_for_site(site, target_key, res_gpp, res_nee)
        if spred is None:
            continue
        modis_dict[site] = [spred, targets, tmsp]
        ax_max = np.max([spred, targets])
        ax_min = np.min([spred, targets])
        fig, ax = newfig(0.9,ratio=1.0)
        ax.scatter(targets, spred, s=3)
        ax.set_xlabel('Target')
        ax.set_ylabel('Prediction')
        ax.text(0.1, 0.9, site,
          horizontalalignment='left',
          verticalalignment='top',
          transform = ax.transAxes)
        ax.plot([ax_min, ax_max], [ax_min, ax_max], color='k',
             lw=2)
        fig.show()

        fig, ax = newfig(2.0, ratio=0.3)
        ax.plot(range(len(targets)), targets, lw=1, color='k',
             marker='o', ms=1)
        ax.plot(range(len(spred)), spred, lw=1,
             marker='o', ms=1)
        ax.text(0.1, 0.9, site,
          horizontalalignment='left',
          verticalalignment='top',
        transform = ax.transAxes)
     #   ax.set_xlim(1600, 1800)
        fig.show()

        month_values_pred = []
        temp_values_pred = [] 
        month_values_truth = []
        temp_values_truth = [] 
        c_month = tmsp[0].month
        for c, i in enumerate(tmsp):
            if i.month != c_month:
                c_month = i.month
                month_values_pred.append(np.mean(temp_values_pred))
                month_values_truth.append(np.mean(temp_values_truth))
                temp_values_truth = [] 
                temp_values_pred = []
            temp_values_pred.append(spred[c])
            temp_values_truth.append(targets[c])
        fig, ax = newfig(0.9)
        ax.plot(range(len(month_values_truth)), month_values_truth, color='k',
             marker='o')
        ax.plot(range(len(month_values_pred)), month_values_pred, color='b',
         marker='o')
        ax.text(0.0, 0.9, site,
          horizontalalignment='left',
          verticalalignment='top',
          transform = ax.transAxes)
        fig.show()

grassland


/home/b/b309233/.local/lib/python3.9/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


Use AT-Neu at (11.3175, 47.11667) (UTC Offset 2.0)
single positional indexer is out-of-bounds
Use CH-Cha at (8.41044, 47.21022) (UTC Offset 2.0)
Use CH-Fru at (8.53778, 47.11583) (UTC Offset 2.0)
Use IT-MBo at (11.04583, 46.01468) (UTC Offset 2.0)
Use IT-Tor at (7.57806, 45.84444) (UTC Offset 2.0)


/home/b/b309233/.local/lib/python3.9/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


Use DE-Gri at (13.51253, 50.94947) (UTC Offset 2.0)
Use DE-RuR at (6.30413, 50.62191) (UTC Offset 2.0)


...

Use IT-Col at (13.58814, 41.84936) (UTC Offset 2.0)
7683
0.6776018059183233
25.86183314459042
-----
13.791124437198413
(array([1.38761805e-01, 4.06058066e+02]), array([[ 1.41364635e-05, -5.90416119e-02],
       [-5.90416119e-02,  2.54038298e+02]])) (array([0.09924171, 1.56798315]), array([[ 1.17422639e-05, -1.46580458e-04],
       [-1.46580458e-04,  2.56688517e-03]]))
shrubland


/home/b/b309233/.local/lib/python3.9/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


Use IT-Noe at (8.15169, 40.60618) (UTC Offset 2.0)


/home/b/b309233/.local/lib/python3.9/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


Use ES-Amo at (-2.25232, 36.83361) (UTC Offset 2.0)
single positional indexer is out-of-bounds
Use ES-LJu at (-2.75212, 36.92659) (UTC Offset 2.0)
3189
0.3173540169060468
5.653091486630461
-----
2.842767044823156
(array([1.20329768e-01, 3.86169388e+02]), array([[ 7.74499011e-05, -3.49892739e-01],
       [-3.49892739e-01,  1.62523785e+03]])) (array([ 0.08743039, -0.07542279]), array([[ 5.92566328e-06, -1.02738050e-04],
       [-1.02738050e-04,  2.05644850e-03]]))


In [14]:
with open('/home/b/b309233/fit_res_modis.pickle', 'w+b') as ofile:
    pickle.dump(fit_res_dict_modis, ofile)

In [32]:
for i in sorted(fit_res_dict_modis.keys()):
    print(i, fit_res_dict_modis[i][0][0])

1 [1.86970869e-01 4.84823741e+02]
2 [1.38761805e-01 4.06058066e+02]
3 [1.46116531e-01 4.55775492e+02]
4 [1.20329768e-01 3.86169388e+02]
6 [1.13850909e-01 5.79096813e+02]
7 [1.30562308e-01 4.97523237e+02]


In [ ]:
for key in modis_dict.keys():
    print(key)
    inds1 = []
    inds2 = []3
    for c, t in enumerate(viirs_dict[key][2]):
        ti = np.where(t==modis_dict[key][2])[0]
        if len(ti)>0:
            inds1.append(c)
            inds2.append(ti[0])
    fig = plt.figure(figsize=figsize(2.0, ratio=0.3))
    ax1 = fig.add_axes((0.,0, 0.31, 1.0))
    ax2 = fig.add_axes((0.33,0, 0.31, 1.0))
    ax3 = fig.add_axes((0.66,0, 0.31, 1.0))
    ax1.scatter(modis_dict[key][1][inds2],modis_dict[key][0][inds2], color='k', s=5)
    ax2.scatter(viirs_dict[key][1][inds1],viirs_dict[key][0][inds1], color='k', s=5)
    ax2.set_ylim(ax1.get_ylim())
    ax2.set_yticklabels([])
    ax3.hist(viirs_dict[key][0][inds1] - modis_dict[key][0][inds2], bins=30, histtype='step')
    ax3.yaxis.tick_right()
    ax1.text(0.0, 1.01, key,
          horizontalalignment='left',
          verticalalignment='bottom',
          transform = ax1.transAxes)
    fig.show()